In [ ]:
# penguin_logistic_tf_strict.py

import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load dataset
penguins = sns.load_dataset('penguins')
penguins.dropna(inplace=True)

# Features and binary target (Adelie vs rest)
X = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].values
y = (penguins['species'] == 'Adelie').astype(int).values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ============================================
# 1️⃣ Reference: Scikit-learn logistic regression
# ============================================
lr_model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
lr_model.fit(X_train, y_train)

print("=== Scikit-learn Logistic Regression ===")
for f, c in zip(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], lr_model.coef_[0]):
    print(f"{f}: {c:.6f}")
print(f"Intercept: {lr_model.intercept_[0]:.6f}\n")

# ============================================
# 2️⃣ TensorFlow: equivalent logistic regression
# ============================================

# Define custom initializer (zeros)
zero_init = tf.keras.initializers.Zeros()

tf_model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(
        1,
        activation='sigmoid',
        kernel_initializer=zero_init,
        bias_initializer=zero_init
    )
])

# SGD optimizer (small learning rate, no momentum)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
tf_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train for enough epochs to converge
tf_model.fit(X_train, y_train, epochs=10000, verbose=0)

# Get weights and bias
weights, bias = tf_model.layers[0].get_weights()

print("=== TensorFlow (SGD, zero init) Logistic Regression ===")
for f, w in zip(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], weights.flatten()):
    print(f"{f}: {w:.6f}")
print(f"Bias (intercept): {bias[0]:.6f}\n")

# Predictions and manual check
y_prob_lr = lr_model.predict_proba(X_test)[:, 1]
y_prob_tf = tf_model.predict(X_test).flatten()

# Manual calculation using learned weights
def manual_sigmoid(x):
    return 1 / (1 + np.exp(-(np.dot(x, weights.flatten()) + bias[0])))

y_prob_manual = np.array([manual_sigmoid(row) for row in X_test])

# Compare
diff = np.max(np.abs(y_prob_lr - y_prob_tf))
print(f"Max difference in predicted probabilities (sklearn vs TF): {diff:.8f}")

# Show a few examples
df = pd.DataFrame({
    "Prob_sklearn": y_prob_lr[:10],
    "Prob_TF": y_prob_tf[:10],
    "Prob_manual": y_prob_manual[:10]
})
print("\nFirst 10 probability comparisons:")
print(df)


=== Scikit-learn Logistic Regression ===
bill_length_mm: -4.397080
bill_depth_mm: 7.546939
flipper_length_mm: 0.013733
body_mass_g: 0.005669
Intercept: 31.659166

